### 1.导入相关包

In [1]:
import time
from matplotlib import pyplot as plt
import numpy as np
import mxnet as mx
from mxnet import autograd, gluon
import gluoncv as gcv
from gluoncv.utils import download, viz

### 2.自定义VOCLike装载自定义数据集

In [2]:
from gluoncv.data import VOCDetection
class VOCLike(VOCDetection):
    CLASSES = ['tiger_beetle', 'scarab','stinkbug','moth']
    def __init__(self, root, splits, transform=None, index_map=None, preload_label=True):
        super(VOCLike, self).__init__(root, splits, transform, index_map, preload_label)


gpu_ctx = mx.gpu(0)

In [3]:
with mx.Context(gpu_ctx):

    dataset = VOCLike(root='~/.mxnet/datasets/voc', splits=((2007, 'trainval'),))
    print('length of dataset:', len(dataset))
    print('label example:')
    print(dataset[0][1])

length of dataset: 378
label example:
[[ 326.  832.  464.  968.    3.    0.]
 [ 856. 1809.  982. 1953.    3.    0.]
 [ 509. 1785.  638. 1953.    3.    0.]
 [ 891. 1979. 1067. 2103.    3.    0.]
 [ 956. 1824. 1282. 2091.    3.    0.]
 [1214. 2291. 1323. 2441.    3.    0.]
 [ 782. 2294. 1012. 2447.    3.    0.]]


In [4]:
VOCLike.CLASSES

['tiger_beetle', 'scarab', 'stinkbug', 'moth']

In [ ]:
# net = gcv.model_zoo.get_model('faster_rcnn_resnet50_v1b_custom', classes=['tiger_beetle', 'scarab', 'stinkbug', 'moth'], pretrained=False)


In [5]:
with mx.Context(gpu_ctx):

    net = gcv.model_zoo.get_model('ssd_512_resnet50_v1_voc', pretrained=True)
# net = gcv.model_zoo.get_model('faster_rcnn_resnet50_v1b_voc', pretrained=True)

/root/.local/lib/python3.5/site-packages/mxnet/gluon/block.py:421: UserWarning: load_params is deprecated. Please use load_parameters.
  warnings.warn("load_params is deprecated. Please use load_parameters.")


In [6]:
net.reset_class(['tiger_beetle', 'scarab', 'stinkbug', 'moth'])

In [7]:
with mx.Context(gpu_ctx):

    net = gcv.model_zoo.get_model('ssd_512_resnet50_v1_custom', ctx=gpu_ctx, classes = VOCLike.CLASSES,
        pretrained_base=False, transfer='voc')

#  gpu_ctx
# net = gcv.model_zoo.get_model('faster_rcnn_resnet50_v1b_custom', ctx=mx.gpu(0), classes = VOCLike.CLASSES,
#     pretrained_base=False, transfer='voc')

/root/.local/lib/python3.5/site-packages/mxnet/gluon/block.py:421: UserWarning: load_params is deprecated. Please use load_parameters.
  warnings.warn("load_params is deprecated. Please use load_parameters.")


### 3.finetune训练新网络

In [ ]:
# from gluoncv.data.transforms.presets.rcnn import FasterRCNNDefaultTrainTransform

In [8]:
def get_dataloader(net, train_dataset, data_shape, batch_size, num_workers):
    from gluoncv.data.batchify import Tuple, Stack, Pad
    from gluoncv.data.transforms.presets.ssd import SSDDefaultTrainTransform
    width, height = data_shape, data_shape
    # use fake data to generate fixed anchors for target generation
    with autograd.train_mode():
        _, _, anchors = net(mx.nd.zeros((1, 3, height, width)))
    batchify_fn = Tuple(Stack(), Stack(), Stack())  # stack image, cls_targets, box_targets
    train_loader = gluon.data.DataLoader(
        train_dataset.transform(SSDDefaultTrainTransform(width, height, anchors)),
        batch_size, True, batchify_fn=batchify_fn, last_batch='rollover', num_workers=num_workers)
    return train_loader




In [ ]:
# net.collect_params().reset_ctx(ctx)

In [ ]:
# with mx.context.Context(mx.gpu(0)):
#     gpu_arr = mx.nd.ones((2,3))



In [ ]:
# mx.context.current_context()

In [9]:
# net.collect_params().reset_ctx(gpu_ctx)

with mx.Context(gpu_ctx):

    train_data = get_dataloader(net, dataset, 512, 16, 0)

In [10]:
try:
    a = mx.nd.zeros((1,), ctx=mx.gpu(0))
    ctx = gpu_ctx
except:
    ctx = [mx.cpu()]

In [11]:
with mx.Context(gpu_ctx):
    
    net.collect_params().reset_ctx(gpu_ctx)  # gpu_ctx
    
    trainer = gluon.Trainer(
        net.collect_params(), 'sgd',
        {'learning_rate': 0.001, 'wd': 0.0005, 'momentum': 0.9})

    mbox_loss = gcv.loss.SSDMultiBoxLoss()
    ce_metric = mx.metric.Loss('CrossEntropy')
    smoothl1_metric = mx.metric.Loss('SmoothL1')


    for epoch in range(0, 10):
        ce_metric.reset()
        smoothl1_metric.reset()
        tic = time.time()
        btic = time.time()
        net.hybridize(static_alloc=True, static_shape=True)
        

        for i, batch in enumerate(train_data):
            batch_size = batch[0].shape[0]
            data = gluon.utils.split_and_load(batch[0], ctx_list=ctx, batch_axis=0)
            cls_targets = gluon.utils.split_and_load(batch[1], ctx_list=ctx, batch_axis=0)
            box_targets = gluon.utils.split_and_load(batch[2], ctx_list=ctx, batch_axis=0)
            with autograd.record():
                cls_preds = []
                box_preds = []
                for x in data:
                    cls_pred, box_pred, _ = net(x)
                    cls_preds.append(cls_pred)
                    box_preds.append(box_pred)
                sum_loss, cls_loss, box_loss = mbox_loss(
                    cls_preds, box_preds, cls_targets, box_targets)
                autograd.backward(sum_loss)
            # since we have already normalized the loss, we don't want to normalize
            # by batch-size anymore
            trainer.step(1)
            ce_metric.update(0, [l * batch_size for l in cls_loss])
            smoothl1_metric.update(0, [l * batch_size for l in box_loss])
            name1, loss1 = ce_metric.get()
            name2, loss2 = smoothl1_metric.get()
            if i % 20 == 0:
                print('[Epoch {}][Batch {}], Speed: {:.3f} samples/sec, {}={:.3f}, {}={:.3f}'.format(
                    epoch, i, batch_size/(time.time()-btic), name1, loss1, name2, loss2))
            btic = time.time()

TypeError: object of type 'Context' has no len()

In [ ]:
net.save_parameters('ssd_512_resnet50_v1_voc.insect.params')

###  4.读取网络参数，run测试集并出结果

In [ ]:
# test_url = 'https://raw.githubusercontent.com/zackchase/mxnet-the-straight-dope/master/img/pikachu.jpg'
# download(test_url, 'pikachu_test.jpg')
# net = gcv.model_zoo.get_model('ssd_512_mobilenet1.0_custom', classes=classes, pretrained_base=False)


In [ ]:
net.load_parameters('ssd_512_resnet50_v1_voc.insect.params')

x, image = gcv.data.transforms.presets.ssd.load_test('pikachu_test.jpg', 512)
cid, score, bbox = net(x)
ax = viz.plot_bbox(image, bbox[0], score[0], cid[0], class_names=classes)
plt.show()